# Projeto #3 - Meu primeiro projeto de IA

Antes de começar, leia as [Instruções](https://github.com/thvmm/pos-ds-ia/blob/master/projeto_3/README.md) e os [Critérios de Avaliação](https://github.com/thvmm/pos-ds-ia/blob/master/projeto_3/README.md).


### 1) **(5%)** Qual a base escolhida e qual seu interesse nela (podem ser mais de uma)? Descrição básica do conjunto de dados escolhido pelo aluno (1 parágrafo).

*Indique o link da base no Kaggle e explique em um parágrafo curto por que essa base é interessante para você. Quais são as características básicas da base?*

Ex: Estou trabalhando com o histório de vendas de um varejo e uma outra base do mesmo varejista das suas campanhas de marketing realizadas em veiculos de comunicação.


As bases de dados selecionadas para esse estudo advêm da combinação dos dados de [perfilagem geofísica] (https://en.wikipedia.org/wiki/Well_logging) de 5 furos de sondagem de exploração mineral (fase de pesquisa) com dados da descrição geológica de [testemunhos de sondagem] (https://en.wikipedia.org/wiki/Core_sample). No contexto da mineração esses dados em conjunto as análises geoquímica com compõe a melhor forma de estudarmos e entendermos como a Terra está organizada em subsuperfície e, consequentemente, como os recursos minerais podem ser explorados e explotados de forma sustentável.

A base de dados da perfilagem geofísica pode ser descrita de forma sismples como a medida em função da profundidades das propriedades físicas dos materiais geológicos que estão em subsuperfície ( e.g. densidade, susceptibilidade magnética, resistividade elétrica, radiação etc).

A base de dados da descrição geológica de testemunhos de sondagem consiste na descrição visual das caracteristicas geológicas dos materiais amostrados pelo furo de sondagem (e.g textura, coloração, mineralogia, classificação em [litologias] (https://en.wikipedia.org/wiki/Lithology) etc). 

Como os dados são de propriedade da Vale S.A. informações sensíveis como localização e nome do poço foram suprimidas do dataset.

### 2) **(5%)** Que tipo de problema podemos solucionar com ela? Liste ao menos 3 hipóteses sobre seus dados, caso sua base seja muito restrita converse com os professores antes de seguir.
- Quais são suas hipoteses sobre os dados? O que vc espera descobrir com esse estudo?

Ex: Hipoteses válidas seriam:
- As pessoas fazem mais compras nos dias utéis ou finais de semana?
- Existe diferença entre o padrão de compra de segunda ou terça feira?
- O horário de pico influencia no padrão de compra dos clientes?
- Quais produtos são comprados mais juntos? Existe diferença se observarmos a idade do cliente?
- Qual o efeito de uma campanha de marketing nas vendas?


Algumas hipoteses são:
* Existe diferença significativas de propriedade fisica para os litotipos observados?
* Há alguma variação nos litotipos observados/amostrados pelos diferentes furos?
* Há alguma relação de dependência dos diferentes litotipos com a profundidade?

### 3) **(20%)** Preparação de dados

Hora de deixar a base perfeita para se trabalhar. Limpe os dados, integre com outras fontes e transforme-o para ficar pronto para se trabalhar! No fim, mostre um antes e depois de seu dataframe.

In [ ]:
# Implemente sua análise aqui. Use mais blocos se achar que ficará mais organizado.
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.metrics import confusion_matrix
import seaborn as sns; sns.set()

#Funções úteis

# Função p/ importar o CLV da descrição para df da perfilagem 
def get_desc(log,desc):
  
    furos = list(desc['Furo'].unique()) #lista dos furos descritos
    li = []
   
    for furo in furos:   
        
        df_log_tmp = log[log['FURO'] == furo]
        df_desc_tmp = desc[desc['Furo'] == furo]
       
        for i in df_log_tmp.index:              #percorre todas instancias do df_log
            for j in df_desc_tmp.index:         #percorre todos os intervalos de descrição
                if df_desc_tmp.get_value(j,'De') <= df_log_tmp.get_value(i,'DEPT') < df_desc_tmp.get_value(j,'Ate'):
                    df_log_tmp.set_value(i,'CLV',df_desc.get_value(j,'CLV'))     #atualiza o CLV no df_log a partir do CLV da descrição
        
        li.append(df_log_tmp)
   
    df = pd.concat(li, ignore_index=True)
   
    return df 

def missing_percentage(df):
    total = df.isnull().sum()
    percent = round(df.isnull().mean()*100,2)
    return pd.concat([total, percent], axis=1, keys=['Total','Percent'])
    
#PLOTS

def make_clv_log_plot(df, curves, clv_colors):

    logs = df.sort_values(by='DEPT')             #ordena o furo por profundidade
    
    cmap_clv = colors.ListedColormap(clv_colors[0:len(clv_colors)], 'indexed')

    ztop=logs.DEPT.min(); zbot=logs.DEPT.max()

    cluster=np.repeat(np.expand_dims(logs['CLV_code'].values,1), 100, 1)

    ncurves = len(curves)

    f, ax = plt.subplots(nrows=1, ncols=ncurves+1, figsize=(ncurves * 2, 12))

    for i, curve in enumerate(curves):
        ax[i].plot(logs[curve], logs['DEPT'], '-', color='black')

    im=ax[ncurves].imshow(cluster, interpolation='none', aspect='auto',
                    cmap=cmap_clv,vmin=1,vmax=17)

    divider = make_axes_locatable(ax[ncurves])
    cax = divider.append_axes("right", size="20%", pad=0.05)
    cbar=plt.colorbar(im, cax=cax)
    cbar.set_label((5*' ').join(['AT','CG', 'MD', 'MSD','MS',
                                    'JPC', 'JPF', 'HF', 'RIF',
                                    'HGOF', 'SR', 'HCTF', 'RIS',
                                    'JPS', 'RIC', 'HC', 'RCF']))
    cbar.set_ticks(range(0,1)); cbar.set_ticklabels('')

    for i in range(len(ax)-1):
        ax[i].set_ylim(ztop,zbot)
        ax[i].invert_yaxis()
        ax[i].grid()
        ax[i].locator_params(axis='x', nbins=3)

    for i, curve in enumerate(curves):
        ax[i].set_xlabel(curve)
        ax[i].set_xlim(logs[curve].min(), logs[curve].max())   
    ax[ncurves].set_xlabel('CLV')

    for i in range(len(ax)-1):
        ax[i+1].set_yticklabels([]); 
    
    ax[ncurves].set_xticklabels([])
    f.suptitle('Furo: %s'%logs.iloc[0]['FURO'], fontsize=14,y=0.94)


def compare_clv_log_plot(df, curves, prediction, clv_colors):

    logs = df.sort_values(by='DEPT')             #ordena o furo por profundidade
    
    cmap_clv = colors.ListedColormap(clv_colors[0:len(clv_colors)], 'indexed')

    ztop=logs.DEPT.min(); zbot=logs.DEPT.max()

    cluster1=np.repeat(np.expand_dims(logs['CLV_code'].values,1), 100, 1)
    cluster2=np.repeat(np.expand_dims(logs[prediction+'_code'].values,1), 100, 1)
   
    ncurves = len(curves)

    f, ax = plt.subplots(nrows=1, ncols=ncurves+2, figsize=(ncurves * 2, 12))

    for i, curve in enumerate(curves):
        ax[i].plot(logs[curve], logs['DEPT'], '-', color='black')

    im1 = ax[ncurves].imshow(cluster1, interpolation='none', aspect='auto',
                    cmap=cmap_clv,vmin=1,vmax=17)
    im2 = ax[ncurves+1].imshow(cluster2, interpolation='none', aspect='auto',                           cmap=cmap_clv,vmin=1,vmax=14)

    divider = make_axes_locatable(ax[ncurves+1])
    cax = divider.append_axes("right", size="20%", pad=0.05)
    cbar=plt.colorbar(im1, cax=cax)
    cbar.set_label((5*' ').join(['AT','CG', 'MD', 'MSD','MS',
                                    'JPC', 'JPF', 'HF', 'RIF',
                                    'HGOF', 'SR', 'HCTF', 'RIS',
                                    'JPS', 'RIC', 'HC', 'RCF']))
    cbar.set_ticks(range(0,1)); cbar.set_ticklabels('')

    for i in range(len(ax)-2):
        ax[i].set_ylim(ztop,zbot)
        ax[i].invert_yaxis()
        ax[i].grid()
        ax[i].locator_params(axis='x', nbins=3)

    for i, curve in enumerate(curves):
        ax[i].set_xlabel(curve)
        ax[i].set_xlim(logs[curve].min(), logs[curve].max())   
    ax[ncurves].set_xlabel('CLV')
    ax[ncurves+1].set_xlabel(prediction)

    for i in range(len(ax)-1):
        ax[i+1].set_yticklabels([]); 
    
    ax[ncurves].set_xticklabels([])
    ax[ncurves+1].set_xticklabels([])
    f.suptitle('Furo: %s'%logs.iloc[0]['FURO'], fontsize=14,y=0.94)

In [ ]:
# importação da base de dados de descrição geológica. Os litotipos são a feature CLV
df_desc = pd.read_csv('./data/desc.csv')
df_desc

In [ ]:
#removendo whitespaces no feature Furo
df_desc['Furo'] = df_desc['Furo'].str.strip()

In [ ]:
# importação da base de dados de perfilagem geofísica
df_log = pd.read_csv('./data/log.csv')

In [ ]:
# conferindo os furos em ambas bases de dados
df_desc['Furo'].unique()

In [ ]:
df_log['FURO'].unique() # furos perfilados

Todos os furos perfilados foram descritos

In [ ]:
# Integração das bases de dados de perfilagem geofísica e descrição geológica - Captura do litotipo (CLV) descrito para o intervalo em todas as profundidade de medidas da perfilagem geofísica
df = get_desc(df_log,df_desc)
df

In [ ]:
# salvar em um arquivo CSV
# df.to_csv('./data/data.csv', index = False)
# df = pd.read_csv('./data/data.csv')
# df.info()

### Descrição da base e suas variáveis

Mnemonic|Unit|Description|Variable Class
:--------:|:----:|:-----------|:---
BIT| MM|Bit size|Discrete
CADE|MM|Caliper from DD3|Continuous
CCLF|CPS|Casing Collar Locator|Continuous
CCO1|MM|3-Arm Caliper CO1-GC2|Continuous
CO1C|CPS|Caliper Raw|Continuous
DD3B|CPS|Short Spaced Density Raw|Continuous
DD3C|CPS|Caliper Raw|Continuous
DD3G|CPS|Gamma Ray Raw |Continuous
DD3L|CPS|Long Spaced Density Raw|Continuous
DENB|G/C3|Density Short Spaced|Continuous
DENL|G/C3|Density Long Spaced|Continuous
DEPT|M|Logged depth|Continuous
DNBO|G/C3|Density Short Spaced - Corrected|Continuous
DNLO|G/C3|Density Long Spaced - Corrected|Continuous
FE1|-|-|Continuous
FE2|-|-|Continuous
GC1G|CPS|Gamma Ray Raw |Continuous
GRC1|GAPI|Gamma Ray from GC1-GC2|Continuous
GRDE|API|Gamma Ray from DD3|Continuous
GRDO|API|Gamma Ray from DD3 - Corrected|Continuous
GTMP|DEGC|Borehole Temperature|Continuous
MSS4|SI|Magnetic Susceptbility|Continuous
MSUS|SI|Magnetic Susceptbility|Continuous

In [ ]:
# Análise de valores ausentes
import missingno as msno
for furo in df.FURO.unique():
    msno.matrix(df[df['FURO']== furo],figsize = (12,6))
    plt.title('%s'%furo, fontsize = 20)
    plt.show();


In [ ]:
missing_percentage(df)

In [ ]:
# Remoção de features - Ausentes > 10% (DNLO, FE1, FE2, MSS4 E MSUS) e Conhecimento a priori (BIT, CADE e CCLF) 
df_selected = df[['FURO', 'DEPT', 'CLV','CCO1', 'CO1C', 'DD3B', 'DD3C', 'DD3G', 'DD3L', 'DENB', 'DENL', 'DNBO', 'GC1G', 'GRC1', 'GRDE', 'GRDO', 'GTMP']]
df_selected

In [ ]:
# Remover intervalos sem descrição geológica - Litotipos (CLV)
df_selected.dropna(subset=['CLV'], axis = 0, how = 'any', inplace = True)

In [ ]:
#Criar dicionário para os Litotipos (CLV)
clvDict = {}
i = 1
for clv in df_selected.CLV.unique():
    clvDict[clv] = i
    i+=1
#criar codigo numérico pra CLV

df_selected['CLV_code'] = df_selected.CLV
df_selected = df_selected.replace({"CLV_code": clvDict})

In [ ]:
clvDict

In [ ]:
# Sumário - Estatística Descritiva Categoricas - Original
df.describe(include = [object])

In [ ]:
# Sumário - Estatística Descritiva Categoricas - Prep
df_selected.describe(include = [object])

In [ ]:
# Sumário - Estatística Descritiva numéricas - Original
df.describe().T

In [ ]:
# Sumário - Estatística Descritiva numéricas - Prep
df_selected.describe().T

### 4) **(30%)** Análise

Com seu dado pronto, é hora de fazer a parte mais legal: investigar e responder nossas hipoteses. Elas se confirmam? Em quais cenários? Existe alguma forma de perceber isso no mundo real? Dica: Abuse das técnicas de visualização.

Ex: Ainda no contexto dos exemplos do item 2). Ao investigar a hipotese relacionada ao padrão de compra, percebi que em todos os meses existia uma diferença entre dia util e final de semana. Porém notei que Fevereiro possui um comportamento diferente, o que poderia explicar? Talvez o carnaval e seus feriados.

Ex2: Talvez eu descubra que alguns finais de semana possuam um comportamento diferente dos dias de semana e outros não. Por que não são todos? Nessa investigação você pode acabar descobrindo que os finais de semana que são diferentes, coincidem no fato de serem os primeiros dias utéis, o que pode remeter ao fato de boa parte das empresas realizarem pagamentos nessa parte do mês.

In [ ]:
# Distribuição dos litotipos
sns.catplot(x="CLV", kind="count", data=df_selected, height=5, aspect=2.5 )
plt.xticks(rotation = 30);

Percebe-se um claro desbalanceamento nos litotipos amostrados pelas campanhas de sondagem geológica. Litotipos como HF, JPC e MD em conjunto representam mais de 70% das amostras.

In [ ]:
# Distribuição dos litotipos por furo
sns.catplot(x = "FURO", hue = "CLV", kind = "count", data = df_selected, height = 5, aspect = 2.5)
plt.xticks(rotation = 0);

Observa-se também que não há semelhança na distribuição dos litotipos amostrados pelos diferentes furos. Por exemplo o furo SSD-FD01006 apresenta grande variabilidade de litotipos amostrados quando comparado com o furo SSF-FD00995. Ou seja, em um área de prospecção mineral as populações de litotipos observados variam significativamente com a posição dos furos. Por exemplo, o litotipo JPS possui um numeto pequeno de amostras, sendo observado apenas nos furos  final 998 e 995. Esse último intercepta em praticamente toda sua extensão o litotipo JPC. Os litotipos MS, MSD e MD são amostrado quase que exclusivamente pelo furo final 1038.

In [ ]:
# Caracterização das propriedades físicas por CLV

for log in df_selected.select_dtypes(include=['int64', 'float64']).drop(['DEPT', 'CLV_code'], axis = 1).columns:  #features numéricos exceto DEPT e CLV_code
    sns.boxplot(x = 'CLV', y = log, data = df_selected)
    plt.xticks(rotation=30)
    plt.show();


Interpretando-se os box-plots acima percebe-se que os logs de perfilagens geofísica que possuem distribuições bem-comportada, ou seja, não apresetam muitos outlier, baixa assimetria e variância, individualizando assim os respectivos litotipos (CLV) são: CCO1, DD3B, DENB, DNBO, GRC1, GTMP.

In [ ]:
# Tendência central das distribuições das propriedades físcas por Litotipo (CLV)
df_selected.drop(['CO1C', 'DD3C', 'DD3G', 'DD3L', 'DENL', 
                    'GC1G', 'GRDE', 'GRDO','DEPT', 'CLV_code'], axis = 1).groupby('CLV').agg(['mean', 'median'])

Com relação as aos valores médios e medianas das diferentes propriedades físicas por litotipo pode-se destacar:
* HF e JPC apresentam sistematicamente valores de densidade (DD3B, DENB e DNBO) mais elevados, bem como menores contagens de radiação gamma (GRC1)
* AT aprensetam maiores diâmetros no Caliper (CO1C)
* RCF e RIC apresentam os maiores valores médios e medianas para contagens de radiação gamma (GRC1)
* MD, MSD e RIF apresentam os maiores valores médios de temperatura (GTMP)

In [ ]:
# Vizualização conjunta dos logs da perfilagem e do CLV - Avaliação visual do comportamento dos logs de perfilagem

#Color dict
clv_colors = ['#F4D03F', '#F5B041','#DC7633','#6E2C00',
                '#1B4F72','#2E86C1', '#AED6F1', '#A569BD',
                '#196F3D','#2E28D1', '#000000', '#A599FD',
                '#F4D20F','#14D20F','#F5D70F','#C1D80F','#A4D20F']

clv_labels = ['AT','CG', 'MD', 'MSD', 'MS',
            'JPC', 'JPF', 'HF', 'RIF', 
            'HGOF', 'SR', 'HCTF', 'RIS',
             'JPS', 'RIC', 'HC', 'RCF']

#dicionario p/ mapear as cores a lito (CLV)
clv_color_map = {}
for ind, label in enumerate(clv_labels):
    clv_color_map[label] = clv_colors[ind]

curves = df_selected.select_dtypes(include=['int64', 'float64']).drop(['DEPT', 'CLV_code'], axis = 1).columns

for furo in df_selected['FURO'].unique():
    make_clv_log_plot(df_selected[df_selected['FURO']==furo], curves, clv_colors)

### 5) Modelagem 30%

Agora você terá mais uma oportunidade de mostrar o que você aprendeu durante o módulo 2 quanto a modelagem de dados e criação de modelos. Utilizando os dados preparados na seção 1 e após a análise feita na seção 2 você deverá:
1. **Defina um problema de regressão ou classificação que envolva uma variável dos seus dados.**

***---> Comente brevemente sua decisão aqui.***
Será realizado o problema de classificação dos litotipos (CLV) a partir dos dados de perfilagem geofísica

2. **Realize ao menos 2 técnicas de processamento e seleção de features.**
    * Isto inclui, normalização, PCA, e técnicas de seleção de features como information gain. Seja criativo pois está parte é crucial para seu modelo. Você pode escolhar manualmente as features desde que seja justificada na seção 2 (parte de Análise).


A avaliação visual do comportamento dos diversos logs de perfilagem juntamente com o a descrição do furo permite:
* observar que o único litotipo que tem uma dependencia clara com a profunidade é AT, ocorrendo sistemetaticamente nos primeiros metros de cada furo;
* Não há necessidade de computar se computar métricas de dependencia (correlação/covariancia) para observar difersor logs apresentam comportamento bastante semelhantes, ou seja, são redundantes.

Dessa forma, combinando-se com a avaliação realizada sobre os box-plots, os logs que serão utilizados como features são: **CCO1, DENB, GRC1 e GTMP**

In [ ]:
data = df_selected.drop(['CO1C', 'DD3C', 'DD3B', 'DNBO', 'DD3G', 'DD3L', 'DENL', 'GC1G', 'GRDE', 'GRDO'], axis = 1)
data.dropna(axis = 0, how = 'any', inplace = True)


In [ ]:
for furo in data['FURO'].unique():
    make_clv_log_plot(data[data['FURO']==furo], ['CCO1', 'DENB', 'GRC1', 'GTMP'], clv_colors)

3. **Defina uma métrica para avaliar o seu modelo.**
    * Por exemplo, você pode utilizar MAE (Mean Absolute Error) para um problema de regressão. Ou, F1-Score para um problema de classificação. Há varias métricas, então escolha sabiamente.


***---> Comente brevemente sua decisão aqui.***

A métrica utilizada será a F1-Score, por ser tratar de um problema de classificação com sujeito a grande influência de verdadeiros negativos, o que impactaria diretamente na acurácia

4. **Divida seus dados em 2 conjuntos. Um de treino e outro conjunto de teste.**
    * Treine e otimize seu modelo no conjunto de treino e apenas use o conjunto de teste para apresentar os resultados finais.


In [ ]:
# Separação de um dos furos para teste cego do modelo de classificação (foi escolhido pela minha sobrinha) =)

test_data = data[data['FURO'] == 'SSD-FD00998']
training_data = data[data['FURO'] != 'SSD-FD00998']
# blind_test_data.dtypes
test_data['FURO'].unique()

In [ ]:
# training_data.dtypes
training_data['FURO'].unique()

In [ ]:
X_train = training_data.drop(['DEPT', 'FURO', 'CLV', 'CLV_code'], axis = 1)
y_train = training_data['CLV']
X_test = test_data.drop(['DEPT', 'FURO', 'CLV', 'CLV_code'], axis = 1)
y_test = test_data['CLV']

In [ ]:
training_data.CLV.unique()

5. **Treine um ou mais modelos de ML para o seu problema.**
    * Escolha 1 ou mais tipos de classificadores ou regressores dependendo do seu problema.
    * Por exemplo, TreeClassifier para um problema de classificação. 
    * Use cross-validation e outras técnicas como GridSearch e ou RandomizedSearch para encontrar os melhores parametros para o seu modelo.


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler


rf = RandomForestClassifier()

## Grade de hiperparâmetros - Random Search

# num de arvores na floresta
n_estimators = [int(x) for x in np.linspace(start = 20, stop = 200, num = 10)]
# num de featrures consideradas em cada split
max_features = ['auto', 'sqrt']
# num max de niveis em uma arvore
max_depth = [int(x) for x in np.linspace(5, 50, num = 10)]
max_depth.append(None)
# num min de samples para split de node
min_samples_split = [2, 5, 10]
# num min de sample em cada folha/node
min_samples_leaf = [1, 2, 4]
# metodo de seleção de amostras para treinar cada arvore
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}




In [ ]:
#Treinamento Random Search

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose = 10, random_state = 42, n_jobs = -1)

rf_random.fit(X_train, y_train)

In [ ]:
# Hiperparâmetros ajustados
print(rf_random.best_params_)

In [ ]:
from sklearn import metrics

best_random = rf_random.best_estimator_
y_rf = best_random.predict(X_test)


print("F1-Score Avg (Random Search): %0.2f" % metrics.f1_score(y_test, y_rf, average = 'weighted'))

In [ ]:
# Armazenar predicoes do RF  na base de test

test_data['RF'] = y_rf
test_data['RF_code'] = y_rf
test_data = test_data.replace({'RF_code': clvDict})
test_data['RF_code'] = test_data['RF_code'].astype('int64')

6. **Apresente (com visualizações) os resultados do seu modelo mostrando que ele é melhor do que um baseline não aleatório.**
    * Para o baseline, você pode escolher um modelo bem trivial mas não aleatório. Por exemplo, para um problema de classificação um baseline pode ser a classe mais presente caso o conjunto de dados seja desbalanceado.  Um modelo mais simples também pode ser utilizado como baseline, por exemplo, você escolheu um Random Forest Classifier, e comparou os resultados um Logistic Regression. Você pode até mesmo escolher um modelo de AutoML(como TPOT) como Baseline.

In [ ]:
# Baseline
from sklearn.model_selection import cross_val_score, StratifiedKFold

rf_base = RandomForestClassifier(n_estimators = 10, random_state = 42)
rf_base.fit(X_train, y_train)

y_base = rf_base.predict(X_test)

print("F1-Score (Baseline): %0.2f" % metrics.f1_score(y_test, y_base, average = 'weighted'))

In [ ]:
# Armazenar predicoes do baseline na base de test
test_data['RF_base'] = y_base
test_data['RF_base_code'] = y_base
test_data = test_data.replace({'RF_base_code': clvDict})


In [ ]:
test_data['RF_base_code'] = test_data['RF_base_code'].astype('int64')

In [ ]:
# Visualizar predicoes do modelo RF
compare_clv_log_plot(test_data, ['CCO1', 'DENB', 'GRC1', 'GTMP'], 'RF', clv_colors)

In [ ]:
# Visualizar predicoes do modelo baseline
compare_clv_log_plot(test_data, ['CCO1', 'DENB', 'GRC1', 'GTMP'], 'RF_base', clv_colors)

### 6) Conclusões **10%**

*Partindo das suas hipoteses e investigações, o que você consegue concluir? Suas hipoteses se concretizaram?*

Seguem algumas conclusões:

* Os litotipos apresentam varições de propriedades físicas, de modo geral
    * HF e JPC apresentam sistematicamente valores de densidade (DD3B, DENB e DNBO) mais elevados, bem como menores contagens de radiação gamma (GRC1)
    * AT aprensetam maiores diâmetros no Caliper (CO1C)
    * RCF e RIC apresentam os maiores valores médios e medianas para contagens de radiação gamma (GRC1)
    * MD, MSD e RIF apresentam os maiores valores médios de temperatura (GTMP)

* Observa-se também que não há semelhança na distribuição dos litotipos amostrados pelos diferentes furos. Por exemplo o furo SSD-FD01006 apresenta grande variabilidade de litotipos amostrados quando comparado com o furo SSF-FD00995. Ou seja, em um área de prospecção mineral as populações de litotipos observados variam significativamente com a posição dos furos. Por exemplo, o litotipo JPS possui um numeto pequeno de amostras, sendo observado apenas nos furos final 998 e 995. Esse último intercepta em praticamente toda sua extensão o litotipo JPC. Os litotipos MS, MSD e MD são amostrado quase que exclusivamente pelo furo final 1038.

* Com exceção do litotipo AT, não existe relação de dependência entre os litotipos e a profundidade 
 
 Com relação a classificação dos litotipos a partir dos dados de perfilagem geofisica:
* Random Forest com hiperparâmetros otimizados (Random Search) mostra um aumento significativo de 25% na métrica de validação (F1-Score) com relação  desempenho do Random Forest - Baseline
* A validação visual mostra que apesar dos contatos entre litotipos estarem bem posicionados, apesar da grande quantidade de falsos positivos
* Por ser tratar de um problema com um grande número de classes penso que seria necessário expadir o número de features para obter melhores resultados.